In [1]:
from main import OpenBookQADataset
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = OpenBookQADataset("train_complete.jsonl", tokenizer)

print("Loaded", len(dataset), "training examples.")
sample_encs, label = dataset[0]
print("Example input_ids shape:", sample_encs[0]["input_ids"].shape)
print("Label:", label)


Loaded 4957 training examples.
Example input_ids shape: torch.Size([1, 128])
Label: 3


In [2]:
from main import OpenBookQADataset, CustomBertMC
from transformers import BertTokenizer
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
dataset = OpenBookQADataset("train_complete.jsonl", tokenizer)

batch = [dataset[i] for i in range(4)]
encodings_batch = [item[0] for item in batch]

model = CustomBertMC().to("cpu")
with torch.no_grad():
    logits = model(encodings_batch)

print("Logits shape:", logits.shape)  # should be [4, 4]
print("Logits:", logits)


Logits shape: torch.Size([4, 4])
Logits: tensor([[0.4618, 0.4196, 0.2734, 0.3441],
        [0.2386, 0.2710, 0.1938, 0.1903],
        [0.4033, 0.3238, 0.4280, 0.3636],
        [0.1562, 0.3797, 0.2178, 0.2392]])


In [3]:
from main import train

trained_model = train()


Starting training...
Epoch 1 | Loss: 1268.29
Epoch 2 | Loss: 679.10
Epoch 3 | Loss: 263.41
Validation Accuracy: 0.644 | Inference Time: 54.76s
Training Time: 8691.42s


In [5]:
from main import OpenBookQADataset, evaluate, custom_collate
from transformers import BertTokenizer
from torch.utils.data import DataLoader

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
test_set = OpenBookQADataset("test_complete.jsonl", tokenizer)
test_loader = DataLoader(test_set, batch_size=4, collate_fn=custom_collate)

test_acc, test_infer_time = evaluate(trained_model, test_loader)
print(f"Test Accuracy: {test_acc:.3f} | Test Inference Time: {test_infer_time:.2f}s")